In [124]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Pitching Airfoil 

In [125]:
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from time import time

# Dynamical Systems

In [126]:
from ICARUS.Systems.first_order_system import NonLinearSystem
from ICARUS.Systems.second_order_system import SecondOrderSystem
from ICARUS.Systems.integrate import BackwardEulerIntegrator, ForwardEulerIntegrator, RK4Integrator, RK45Integrator, CrankNicolsonIntegrator, GaussLegendreIntegrator, NewmarkIntegrator

In [127]:
def plot_results(x_data, t_data):
    # Plot the results
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    for key in x_data.keys():
        # Compute cap so that we plot 1000 points at most
        cap = max(1, int(len(t_data[key]) / 1000))
        ax.plot(t_data[key][::cap], x_data[key][:,0][::cap], label=key)
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Displacement (m)")
    ax.legend()


def test_all_integrators(system,x0, t0, t_end, dt0, compare_with_scipy = False):
    if isinstance(system, SecondOrderSystem):
        system2 = system.convert_to_first_order()
        newmark = NewmarkIntegrator(dt0, system, gamma=0.5, beta=0.25)
    else:
        system2 = system
    
    integrator_rk4 = RK4Integrator(dt0, system2)
    integrator_feuler = ForwardEulerIntegrator(dt0, system2)
    integrator_beuler = BackwardEulerIntegrator(dt0, system2)
    integrator_rk45 = RK45Integrator(dt0, system2)
    integrator_gauss_legendre = GaussLegendreIntegrator(dt0, system2, n = 4)
    integrator_crank_nicolson = CrankNicolsonIntegrator(dt0, system2)

    integrators = [
        # integrator_feuler,
        # integrator_rk4,
        integrator_rk45,
        integrator_beuler,
        integrator_gauss_legendre,
        # integrator_crank_nicolson
     ]
    
    if isinstance(system, SecondOrderSystem):
        integrators.append(newmark)

    # Simulate the system using all the integrators
    x_data = {}
    t_data = {}


    for integrator in integrators:
        print(f"Simulating using {integrator.name} integrator")
        time_s = time()
        t_data[integrator.name], x_data[integrator.name] = integrator.simulate(x0, t0, t_end)
        time_e = time()
        print(f"\tSimulated using {integrator.name} integrator")
        print(f"\tTime taken: {time_e - time_s} seconds")

    if compare_with_scipy:
        print("Simulating using scipy RK45")
        time_s = time()
        sol = solve_ivp(system, [t0, t_end], x0, method='RK45', t_eval=np.linspace(0, t_end, 1000),  rtol=1e-6, atol=1e-6)
        t_data["scipy_rk45"] = sol.t
        x_data["scipy_rk45"] = sol.y.T
        time_e = time()
        print(f"\tSimulated using scipy RK45")
        print(f"\tTime taken: {time_e - time_s} seconds")

    plot_results(x_data, t_data)


# Simple Mass-Damper System

In [128]:
# Define a simple m-c-k system
m = 1.0
c = 0.1
k = 1.0


def f(t:float, x: jnp.ndarray) -> jnp.ndarray:
    return jnp.array([
        x[1],                            # x' = v
        -c /m * x[1] - k/m * x[0]        # v' = a = -c/m * v - k/m * x
    ])

# Create the system
system = NonLinearSystem(f)

# Test the integrators
test_all_integrators(system, jnp.array([1.0, 0.0]), 0.0, 100.0, 0.000001, compare_with_scipy = True)

Simulating using RK45 integrator
	Simulated using RK45 integrator
	Time taken: 0.623565673828125 seconds
Simulating using Backward Euler integrator
	Simulated using Backward Euler integrator
	Time taken: 37.49129819869995 seconds
Simulating using Gauss-Legendre integrator
	Simulated using Gauss-Legendre integrator
	Time taken: 7.260361433029175 seconds
Simulating using scipy RK45
	Simulated using scipy RK45
	Time taken: 0.1746962070465088 seconds


# Second Order Systems

In [129]:
# Define a 2nd order system 
m1 = 1.0
c1 = 0.1
k1 = 1.0

m2 = 1.0
c2 = 0.1
k2 = 1.0

def M(t,x):
    return jnp.array(
    [
        [m1, 0],
        [0, m2]
    ])
# M = jnp.array([m])
def C(t,x):
    return jnp.array(
    [
        [c1, 0],
        [0, c2]
    ])
# C = jnp.array([c])

def f_int(t,x):
    return jnp.array(
    [
        [k1*t, -k1],
        [-k1, k1 + k2]
    ])
# f_int = jnp.array([k])

def f_ext(t: float, x: jnp.ndarray) -> jnp.ndarray:
    return jnp.array(
        [0.0, jnp.sin(t)]
    )
# f_ext = lambda t, x: jnp.array([0.0])

system = SecondOrderSystem(M, C, f_int, f_ext)

In [130]:
# Test the integrators
test_all_integrators(system, jnp.array([1.0, 0.0, 0.0, 0.0]), 0.0, 10.0, 0.001, compare_with_scipy = True)

Simulating using RK45 integrator
	Simulated using RK45 integrator
	Time taken: 1.3255565166473389 seconds
Simulating using Backward Euler integrator
	Simulated using Backward Euler integrator
	Time taken: 0.6203367710113525 seconds
Simulating using Gauss-Legendre integrator
	Simulated using Gauss-Legendre integrator
	Time taken: 1.4833793640136719 seconds
Simulating using Newmark integrator
	Simulated using Newmark integrator
	Time taken: 0.20244765281677246 seconds
Simulating using scipy RK45
	Simulated using scipy RK45
	Time taken: 0.20554041862487793 seconds


# Node

In [131]:
class Node:
    idx = 0
    var_global_idx = {}
    """
    A node represents a point that has some degree of freedom (DOF) associated with it.
    Additionally it can have boundary conditions applied to it.
    """
    def __init__(self, degree_of_freedom: dict[str, float], boundary_conditions = None):
        self.dof = len(degree_of_freedom.keys())
        self.variables = degree_of_freedom
        self.boundary_conditions = boundary_conditions

        # Node index
        self.index = Node.idx
        Node.idx += 1

        # For each degree of freedom, assign a global index if it is not already assigned
        for key in degree_of_freedom.keys():
            if key not in Node.var_global_idx:
                Node.var_global_idx[key] = len(Node.var_global_idx)
        

    def __str__(self):
        return f"Node {self.index} with {self.dof} degrees of freedom"
    
    @staticmethod
    def get_global_idx(key)-> int:
        return Node.var_global_idx[key]
    
    @staticmethod
    def get_global_dof_name(idx):
        for key, value in Node.var_global_idx.items():
            if value == idx:
                return key
        return None
    
# Create a simple 3 node system
node1 = Node({"x1": 0.0, "y1": 0.0, "theta1": 0.0})
node2 = Node({"x2": 0.0, "y2": 0.0, "theta2": 0.0})
node3 = Node({"x3": 0.0, "y3": 0.0, "theta3": 0.0})


In [132]:
class NodeConnection:
    """
    A node connection represents how two nodes are connected to each other 
    and how they are connected to the global coordinate system.    
    """
    def __init__(self, node_1: Node, node_2: Node, phi: float):
        self.node_1 = node_1
        self.node_2 = node_2
        self.nodes = [node_1, node_2]
        self.phi = phi

        # Check that the nodes are valid
        if not isinstance(node_1, Node) or not isinstance(node_2, Node):
            raise ValueError("node_1 and node_2 must be instances of the Node class")
        
        # Make a set of the degrees of freedom
        dof = set()
        for degree in self.node_1.variables.keys():
            dof.add(degree)
        for degree in self.node_2.variables.keys():
            dof.add(degree)
            
        self.dof = len(dof)
        self.variables = dof

        # Note the global indices of the degrees of freedom
        self.global_indices = [Node.var_global_idx[d] for d in dof]

# Connect node 1 to node 2
# Compute the angle of the connection
phi = np.arctan2(node2.variables["y2"] - node1.variables["y1"], node2.variables["x2"] - node1.variables["x1"])
conn1 = NodeConnection(node1, node2, phi)

# Connect node 2 to node 3
# Compute the angle of the connection
phi = np.arctan2(node3.variables["y3"] - node2.variables["y2"], node3.variables["x3"] - node2.variables["x2"])
conn2 = NodeConnection(node2, node3, phi)

# Finite Elements

In [187]:
class FiniteElement:
    def __init__(self, E, A, L,I, rho, node_1: Node, node_2: Node, node_connection: NodeConnection):
        self.E = E
        self.A = A
        self.L = L
        self.I = I
        self.rho = rho
        
        # Check that the nodes are valid
        if not isinstance(node_1, Node) or not isinstance(node_2, Node):
            raise ValueError("node_1 and node_2 must be instances of the Node class")
        
        # Check that the node connection is valid
        if not isinstance(node_connection, NodeConnection):
            raise ValueError("node_connection must be an instance of the NodeConnection class")
        
        self.node_1 = node_1
        self.node_2 = node_2
        self.node_connection = node_connection
        self.nodes = [node_1, node_2]
        
        # Register the finite element degrees of freedom
        self.dof = node_connection.dof
        self.variables = node_connection.variables

        # Register the global indices of the degrees of freedom
        self.global_indices = node_connection.global_indices

    def local_M(self,t,x):
        return (
            self.rho * self.A * self.L / 420
        )* jnp.array([
            [140, 0, 0,70 ,0,0],
            [0, 156, 22*self.L,0,54,-13*self.L],
            [0,22*self.L,4*self.L**2,0,13*self.L,-3*self.L**2],
            [70,0,0,140,0,0],
            [0,54,13*self.L,0,156,-22*self.L],
            [0,-13*self.L,-3*self.L**2,0,-22*self.L,4*self.L**2]
        ])
    
    def local_K(self,t,x):
        return jnp.array([
            [self.E*self.A/self.L, 0, 0, -self.E*self.A/self.L, 0, 0],
            [0, 12*self.E*self.I/self.L**3, 6*self.E*self.I/self.L**2, 0, -12*self.E*self.I/self.L**3, 6*self.E*self.I/self.L**2],
            [0, 6*self.E*self.I/self.L**2, 4*self.E*self.I/self.L, 0, -6*self.E*self.I/self.L**2, 2*self.E*self.I/self.L],
            [-self.E*self.A/self.L, 0, 0, self.E*self.A/self.L, 0, 0],
            [0, -12*self.E*self.I/self.L**3, -6*self.E*self.I/self.L**2, 0, 12*self.E*self.I/self.L**3, -6*self.E*self.I/self.L**2],
            [0, 6*self.E*self.I/self.L**2, 2*self.E*self.I/self.L, 0, -6*self.E*self.I/self.L**2, 4*self.E*self.I/self.L]
        ])
    
    def local_C(self,t,x):
        return jnp.array([
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0]
        ])
    
    @property
    def R(self):
        phi = self.node_connection.phi
        return jnp.array([
            [jnp.cos(phi), jnp.sin(phi), 0, 0, 0, 0],
            [-jnp.sin(phi), jnp.cos(phi), 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0],
            [0, 0, 0, jnp.cos(phi), jnp.sin(phi), 0],
            [0, 0, 0, -jnp.sin(phi), jnp.cos(phi), 0],
            [0, 0, 0, 0, 0, 1]
        ])

    def M(self,t,x):
        return self.R.T @ self.local_M(t,x) @ self.R
    
    def K(self,t,x):
        return self.R.T @ self.local_K(t,x) @ self.R
    
    def C(self,t,x):
        return self.R.T @ self.local_C(t,x) @ self.R
    
    def __str__(self):
        return f"Finite Element with {self.dof} degrees of freedom.\n\tNode 1: {self.node_1}\n\tNode 2: {self.node_2}"
    
# Create two finite elements
E = 1.0
A = 1.0
L = 1.0
I = 1.0 
rho = 1.0

element1 = FiniteElement(E, A, L, I, rho, node1, node2, conn1)
element2 = FiniteElement(E, A, L, I, rho, node2, node3, conn2)

In [190]:
import jax.numpy as jnp
from functools import partial
from jax import jit

class NodalSystem:
    """
    An nodal system represents a collection of finite elements that are connected to each other. 
    """
    def __init__(self, elements: list[FiniteElement]):
        self.elements = elements
        
        # Register the degrees of freedom using a set
        dof = set()
        for element in elements:
            for variable in element.variables:
                dof.add(variable)
        self.dof = len(dof)

    @partial(jit, static_argnums=(0))
    def M(self,t, x):
        # Construct the M matrix by adding the local M matrices. 
        # The global M matrix is a block matrix of the local M matrices
        # The matrix should be sparse
        M = jnp.zeros((self.dof, self.dof))

        # To construct the matrix loop over all the elements. 
        for element in self.elements:
            # Get each elements K matrix expressed in the global coordinate system
            sub_M =  element.M(t,x)
            for node in element.nodes:
                for local_row, dof in enumerate(node.variables.keys()):
                    global_row = Node.get_global_idx(dof)
                    # Set the mask true for each idx in the Nodes global indices
                    columns = jnp.array([ i  for i in element.global_indices])
                    # M[idx, mask] += sub_M[local_idx, :]
                    M = M.at[global_row, columns].add(sub_M[local_row, :])
        return M
    
    @partial(jit, static_argnums=(0))
    def K(self, t, x):
        # Construct the K matrix by adding the local K matrices. 
        # The global K matrix is a block matrix of the local K matrices
        # The matrix should be sparse
        K = jnp.zeros((self.dof, self.dof))

        # To construct the matrix loop over all the elements. 
        for element in self.elements:
            # Get each elements K matrix expressed in the global coordinate system
            sub_K =  element.K(t,x)
            for node in element.nodes:
                for local_row, dof in enumerate(node.variables.keys()):
                    global_row = Node.get_global_idx(dof)
                    # Set the mask true for each idx in the Nodes global indices
                    columns = jnp.array([ i  for i in element.global_indices])
                    # K[idx, mask] += sub_K[local_idx, :]
                    K = K.at[global_row, columns].add(sub_K[local_row,:])
        return K
    
    def C(self,t,x):
        # Construct the C matrix by adding the local C matrices. 
        # The global C matrix is a block matrix of the local C matrices
        # The matrix should be sparse
        C = jnp.zeros((self.dof, self.dof))

        # To construct the matrix loop over all the elements. 
        for element in self.elements:
            # Get each elements C matrix expressed in the global coordinate system
            sub_C =  element.C(t,x)
            for node in element.nodes:
                for local_row, dof in enumerate(node.variables.keys()):
                    global_row = Node.get_global_idx(dof)                    
                    # Set the mask true for each idx in the Nodes global indices
                    columns = jnp.array([ i  for i in element.global_indices])               
                    # C[idx, mask] += sub_C[local_idx, :]
                    C = C.at[global_row, columns].add(sub_C[local_row, :])
        return C
        
sys = NodalSystem([element1, element2])

M = sys.M(0.0, jnp.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))

print('\n'.join([''.join(['{:2f}  '.format(item) for item in row]) 
      for row in M]))

def f_ext(t: float, x: jnp.ndarray) -> jnp.ndarray:
    return jnp.array([
        jnp.sin(2*np.pi/(3)*t),             # theta1
        0.0,                                # x1
        0.0,                                # y1
        0.0,                                # x2
        0.0,                                # y2
        0.0,                                # theta2
        0.0,                                # x3    
        0.0,                                # y3
        0.0                                 # theta3
    ])

0.166667  0.333333  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
0.000000  0.000000  0.371429  0.052381  -0.030952  0.128571  0.000000  0.000000  0.000000  
0.000000  0.000000  0.052381  0.009524  -0.007143  0.030952  0.000000  0.000000  0.000000  
0.166667  0.333333  0.000000  0.000000  0.000000  0.166667  0.000000  0.333333  0.000000  
0.000000  0.000000  0.371429  0.423810  -0.061905  0.128571  0.128571  0.000000  0.052381  
0.000000  0.000000  0.052381  0.061905  -0.014286  0.030952  0.030952  0.000000  0.009524  
0.000000  0.000000  0.000000  0.000000  0.000000  0.166667  0.000000  0.333333  0.000000  
0.000000  0.000000  0.000000  0.371429  -0.030952  0.000000  0.128571  0.000000  0.052381  
0.000000  0.000000  0.000000  0.052381  -0.007143  0.000000  0.030952  0.000000  0.009524  


# Model

In [192]:
system = SecondOrderSystem(sys.M, sys.C, sys.K, f_ext)

# Newmark integrator
newmark = NewmarkIntegrator(0.001, system, gamma=0.5, beta=0.25)
t0 = 0.0
t_end = 10.0
x0 = jnp.zeros(sys.dof*2)
t_data, x_data = newmark.simulate(x0, t0, t_end)
plt.figure()
plt.plot(t_data, x_data[:,0])


In [194]:
x_data.shape 

(10001, 18)